In [ ]:
! pip install langchain_community langchain-openai langchain-anthropic scikit-learn bs4 pandas pyarrow matplotlib lxml langgraph "mcp[cli]"


In [ ]:
from bs4 import BeautifulSoup
import requests
import re
from urllib.parse import urlparse
from langchain_community.document_loaders import RecursiveUrlLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [101]:
def format_boursorama_output(raw_text):
    """
    Reformate le texte brut extrait de Boursorama en un format plus lisible
    tout en conservant plus d'informations
    """
    # Extraire les informations clés
    result = {
        "nom": "",
        "cours": "",
        "devise": "",
        "variation": "",
        "valeur_indicative": "",
        "code_isin": "",
        "symbole": "",
        "marche": "",
        "indicateurs": {},
        "autres_info": {}
    }
    
    # Rechercher le nom de l'action
    if "IQVIA" in raw_text:
        result["nom"] = "IQVIA HOLDINGS"
    if "CEGEDIM" in raw_text:
        result["nom"] = "CEGEDIM HOLDINGS"    
    
    # Rechercher le cours (chiffre avec décimales)
    cours_pattern = re.search(r'HOLDINGS\s*(\d+[,.]\d+)', raw_text)
    if cours_pattern:
        result["cours"] = cours_pattern.group(1)
    
    # Rechercher la devise
    if "USD" in raw_text:
        result["devise"] = "USD"
    
    # Rechercher la variation (pourcentage avec signe)
    variation_pattern = re.search(r'(-?\d+[,.]\d+%)', raw_text)
    if variation_pattern:
        result["variation"] = variation_pattern.group(1)
    
    # Rechercher la valeur indicative
    val_ind_pattern = re.search(r'valeur indicative\s*(\d+[,.]\d+)', raw_text)
    if val_ind_pattern:
        result["valeur_indicative"] = val_ind_pattern.group(1)
    
    # Rechercher le code ISIN
    isin_pattern = re.search(r'(US\d+)', raw_text)
    if isin_pattern:
        result["code_isin"] = isin_pattern.group(1)
    
    # Rechercher le symbole boursier
    symbole_pattern = re.search(r'IQV[^\s]*', raw_text)
    if symbole_pattern:
        result["symbole"] = symbole_pattern.group(0)
    
    # Rechercher le marché
    if "NYSE" in raw_text:
        result["marche"] = "NYSE"
    
    # Extraire les indicateurs clés
    indicateurs = {
        "ouverture": re.search(r'ouverture\s*(\d+[,.]\d+)', raw_text),
        "clôture veille": re.search(r'clôture veille\s*(\d+[,.]\d+)', raw_text),
        "plus haut": re.search(r'\+ haut\s*(\d+[,.]\d+)', raw_text),
        "plus bas": re.search(r'\+ bas\s*(\d+[,.]\d+)', raw_text),
        "volume": re.search(r'volume\s*(\d+\s*\d*)', raw_text),
        "capital échangé": re.search(r'capital échangé\s*(\d+[,.]\d+%)', raw_text),
        "valorisation": re.search(r'valorisation\s*(\d+\s*\d*\s*MUSD)', raw_text),
        "capitalisation": re.search(r'capi. boursière[^0-9]*(\d+\s*\d*\s*M)', raw_text)
    }
    
    for key, match in indicateurs.items():
        if match:
            result["indicateurs"][key] = match.group(1)
    
    # Formater le résultat avec plus d'informations
    formatted_output = f"┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓\n"
    formatted_output += f"┃ {result['nom']} - COTATION BOURSORAMA             ┃\n"
    formatted_output += f"┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛\n\n"
    
    # Informations principales
    formatted_output += f"📈 COURS ACTUEL: {result['cours']} {result['devise']} ({result['variation']})\n"
    formatted_output += f"   Valeur indicative: {result['valeur_indicative']} EUR\n\n"
    
    # Informations de référence
    formatted_output += f"🏛️ RÉFÉRENCES:\n"
    formatted_output += f"   • Symbole: {result['symbole']}\n"
    formatted_output += f"   • ISIN: {result['code_isin']}\n"
    formatted_output += f"   • Marché: {result['marche']} (données temps différé)\n\n"
    
    # Indicateurs de marché
    formatted_output += f"📊 INDICATEURS DE MARCHÉ:\n"
    formatted_output += f"   • Ouverture: {result['indicateurs'].get('ouverture', 'N/A')}\n"
    formatted_output += f"   • Clôture veille: {result['indicateurs'].get('clôture veille', 'N/A')}\n"
    formatted_output += f"   • Plus haut: {result['indicateurs'].get('plus haut', 'N/A')}\n"
    formatted_output += f"   • Plus bas: {result['indicateurs'].get('plus bas', 'N/A')}\n\n"
    
    # Volumes et capitalisation
    formatted_output += f"📉 VOLUMES ET CAPITALISATION:\n"
    formatted_output += f"   • Volume: {result['indicateurs'].get('volume', 'N/A')}\n"
    formatted_output += f"   • Capital échangé: {result['indicateurs'].get('capital échangé', 'N/A')}\n"
    formatted_output += f"   • Valorisation: {result['indicateurs'].get('valorisation', 'N/A')}\n"
    formatted_output += f"   • Capitalisation: {result['indicateurs'].get('capitalisation', 'N/A')}\n"
    
    return formatted_output

In [82]:
# A partir des données brutes 

In [96]:
def format_boursorama_output(raw_text):
    """
    Reformate le texte brut extrait de Boursorama en un format plus lisible
    """
    # Extraire les informations clés
    result = {
        "nom": "",
        "cours": "",
        "devise": "",
        "variation": "",
        "indicateurs": {}
    }
    
    # Extraire le nom directement depuis stock_name
    stock_name_match = re.search(r'stock_name:\s*Cours([A-Z][A-Za-z0-9\s&]+)', raw_text)
    if stock_name_match:
        result["nom"] = stock_name_match.group(1).strip()
    
    # Rechercher le cours (chiffre avec décimales)
    cours_pattern = re.search(r'([0-9]+[,.][0-9]+)EUR', raw_text)
    if cours_pattern:
        result["cours"] = cours_pattern.group(1)
    
    # Rechercher la devise
    if "EUR" in raw_text:
        result["devise"] = "EUR"
    elif "USD" in raw_text:
        result["devise"] = "USD"
    
    # Rechercher la variation (pourcentage avec signe)
    variation_pattern = re.search(r'([+-]?[0-9]+[,.][0-9]+%)', raw_text)
    if variation_pattern:
        result["variation"] = variation_pattern.group(1)
    
    # Extraire les indicateurs clés
    indicateurs = {
        "ouverture": re.search(r'ouverture\s*([0-9]+[,.][0-9]+)', raw_text),
        "clôture veille": re.search(r'clôture veille\s*([0-9]+[,.][0-9]+)', raw_text),
        "plus haut": re.search(r'\+ haut\s*([0-9]+[,.][0-9]+)', raw_text),
        "plus bas": re.search(r'\+ bas\s*([0-9]+[,.][0-9]+)', raw_text),
        "volume": re.search(r'volume\s*([0-9\s]+)', raw_text),
        "capital échangé": re.search(r'capital échangé\s*([0-9]+[,.][0-9]+%)', raw_text),
        "valorisation": re.search(r'valorisation\s*([0-9]+\s*[A-Z]EUR)', raw_text)
    }
    
    for key, match in indicateurs.items():
        if match:
            result["indicateurs"][key] = match.group(1)
    
    # Extraire le code ISIN
    isin_match = re.search(r'([A-Z]{2}[0-9]{10})', raw_text)
    if isin_match:
        result["isin"] = isin_match.group(1)
    
    # Extraire le symbole
    symbol_match = re.search(r'([A-Z]{2,5})[A-Za-z]+ Paris', raw_text)
    if symbol_match:
        result["symbole"] = symbol_match.group(1)
    
    # Extraire le marché
    if "Euronext Paris" in raw_text:
        result["marche"] = "Euronext Paris"
    elif "NYSE" in raw_text:
        result["marche"] = "NYSE"
    
    # Formater le résultat avec plus d'informations
    formatted_output = f" ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓\n"
    formatted_output += f"┃ {result['nom']} - COTATION BOURSORAMA          ┃\n"
    formatted_output += f"┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛\n\n"
    
    # Informations principales
    formatted_output += f"📈 COURS ACTUEL: {result['cours']} {result['devise']} ({result['variation']})\n\n"
    
    # Informations de référence
    formatted_output += f"🏛️ RÉFÉRENCES:\n"
    formatted_output += f"   • Symbole: {result.get('symbole', 'N/A')}\n"
    formatted_output += f"   • ISIN: {result.get('isin', 'N/A')}\n"
    formatted_output += f"   • Marché: {result.get('marche', 'N/A')}\n\n"
    
    # Indicateurs de marché
    formatted_output += f"📊 INDICATEURS DE MARCHÉ:\n"
    formatted_output += f"   • Ouverture: {result['indicateurs'].get('ouverture', 'N/A')}\n"
    formatted_output += f"   • Clôture veille: {result['indicateurs'].get('clôture veille', 'N/A')}\n"
    formatted_output += f"   • Plus haut: {result['indicateurs'].get('plus haut', 'N/A')}\n"
    formatted_output += f"   • Plus bas: {result['indicateurs'].get('plus bas', 'N/A')}\n\n"
    
    # Volumes et capitalisation
    formatted_output += f"📉 VOLUMES ET CAPITALISATION:\n"
    formatted_output += f"   • Volume: {result['indicateurs'].get('volume', 'N/A')}\n"
    formatted_output += f"   • Capital échangé: {result['indicateurs'].get('capital échangé', 'N/A')}\n"
    formatted_output += f"   • Valorisation: {result['indicateurs'].get('valorisation', 'N/A')}\n"
    
    return formatted_output

In [ ]:
# the good 

In [102]:
def boursorama_extractor(html):
    """
    Extracteur spécifique pour Boursorama, utilisant les classes et IDs spécifiques
    identifiés dans le HTML de Boursorama, avec extraction améliorée des données
    """
    soup = BeautifulSoup(html, "lxml")
    
    extracted_content = {}
    
    # Essayer d'extraire les données JSON des attributs data-faceplate
    faceplate_data = soup.find(attrs={"data-faceplate": True})
    if faceplate_data:
        try:
            data_init = faceplate_data.get("data-faceplate-is-init", "{}")
            if data_init:
                import json
                stock_data = json.loads(data_init)
                
                # Extraire les informations clés du JSON
                for key in ["symbol", "high", "low", "previousClose", "totalVolume", "last"]:
                    if key in stock_data:
                        extracted_content[f"json_{key}"] = stock_data[key]
        except Exception as e:
            extracted_content["json_error"] = str(e)
    
    # Extraire le titre/nom de l'action
    title_element = soup.find("h1")
    if title_element:
        extracted_content["stock_name"] = title_element.get_text(strip=True)
    
    # Trouver les éléments spécifiques à Boursorama
    target_elements = [
        {"tag": "main", "id": "main-content", "key": "main_content"},
        {"tag": "section", "class": "l-quotepage", "key": "quote_data"},
        {"tag": "div", "class": "c-faceplate", "key": "faceplate"},
        {"tag": "div", "class": "l-page__content", "key": "page_content"},
        {"tag": "section", "class": "l-header__gigaban", "key": "header_data"}
    ]
    
    for element in target_elements:
        tag = element["tag"]
        key = element["key"]
        
        if "id" in element and element["id"]:
            found = soup.find(tag, id=element["id"])
        else:
            found = soup.find(tag, class_=element["class"])
            
        if found:
            extracted_content[key] = found.get_text(strip=True)
    
    # Extraire les éléments spécifiques de prix et données financières
    price_element = soup.find(class_=lambda c: c and "c-instrument--last" in str(c))
    if price_element:
        extracted_content["price"] = price_element.get_text(strip=True)
    
    currency_element = soup.find(class_=lambda c: c and "c-instrument--currency" in str(c))
    if currency_element:
        extracted_content["currency"] = currency_element.get_text(strip=True)
    
    variation_element = soup.find(class_=lambda c: c and "c-instrument--variation" in str(c))
    if variation_element:
        extracted_content["variation"] = variation_element.get_text(strip=True)
    
    # Extraire les données du tableau d'indicateurs
    indicators = {
        "ouverture": "ouverture",
        "clôture veille": "clôture veille",
        "plus haut": "+ haut",
        "plus bas": "+ bas",
        "volume": "volume",
        "capital échangé": "capital échangé",
        "valorisation": "valorisation"
    }
    
    for key, text in indicators.items():
        indicator_row = soup.find(string=lambda s: s and text in s)
        if indicator_row:
            parent = indicator_row.parent
            value_element = parent.find_next("td") if parent else None
            if value_element:
                extracted_content[f"indicator_{key}"] = value_element.get_text(strip=True)
    
    # Extraire les codes ISIN et symboles
    isin_match = soup.find(string=lambda s: s and re.search(r'[A-Z]{2}\d{10}', s))
    if isin_match:
        isin_code = re.search(r'([A-Z]{2}\d{10})', isin_match)
        if isin_code:
            extracted_content["isin"] = isin_code.group(1)
    
    symbol_element = soup.find(attrs={"data-ist": True})
    if symbol_element:
        extracted_content["symbol"] = symbol_element.get("data-ist")
    
    # Chercher le marché
    market_match = soup.find(string=lambda s: s and any(market in s for market in ["NYSE", "NASDAQ", "EURONEXT"]))
    if market_match:
        for market in ["NYSE", "NASDAQ", "EURONEXT"]:
            if market in market_match:
                extracted_content["market"] = market
                break
    
    # Format result
    formatted_text = ""
    for key, value in extracted_content.items():
        formatted_text += f"{key}:\n{value}\n\n"
    
    return formatted_text if formatted_text else soup.get_text()

In [ ]:
# the bad

In [97]:
def boursorama_extractor(html):
    """
    Extracteur spécifique pour Boursorama, utilisant les classes et IDs spécifiques
    identifiés dans le HTML de Boursorama
    """
    soup = BeautifulSoup(html, "lxml")
    
    extracted_content = {}
    
    # Trouver les éléments spécifiques à Boursorama
    target_elements = [
        {"tag": "main", "id": "main-content", "key": "main_content"},
        {"tag": "section", "class": "l-quotepage", "key": "quote_data"},
        {"tag": "div", "class": "c-faceplate", "key": "faceplate"},
        {"tag": "div", "class": "l-page__content", "key": "page_content"},
        {"tag": "section", "class": "l-header__gigaban", "key": "header_data"}
    ]
    
    for element in target_elements:
        tag = element["tag"]
        key = element["key"]
        
        if "id" in element and element["id"]:
            found = soup.find(tag, id=element["id"])
        else:
            found = soup.find(tag, class_=element["class"])
            
        if found:
            extracted_content[key] = found.get_text(strip=True)
    
    # Chercher spécifiquement les données de prix/cotation
    price_elements = soup.find_all(class_=lambda c: c and any(x in str(c) for x in 
                                                            ["c-instrument", "c-quote", "c-faceplate"]))
    
    for i, el in enumerate(price_elements):
        key = f"financial_data_{i}"
        extracted_content[key] = el.get_text(strip=True)
    
    # Format result
    formatted_text = ""
    for key, value in extracted_content.items():
        formatted_text += f"{key}:\n{value}\n\n"
    
    return formatted_text if formatted_text else soup.get_text()

In [103]:
def financial_site_extractor(html):
    soup = BeautifulSoup(html, "lxml")
    
    # Cherche des conteneurs communs pour les données financières
    content_selectors = [
        {"tag": "div", "class": ["c-faceplate", "main-content", "stock-summary"]},
        {"tag": "main", "class": None},
        {"tag": "div", "id": ["quote-summary", "quote-header", "quote-content"]}
    ]
    
    extracted_content = {}
    
    for selector in content_selectors:
        tag = selector.get("tag", "div")
        # Vérifier si la clé "class" existe ET n'est pas None
        if "class" in selector and selector["class"] is not None:
            for class_name in selector["class"]:
                content = soup.find(tag, class_=class_name)
                if content:
                    extracted_content[class_name] = content.get_text(strip=True)
        # Vérifier si la clé "id" existe ET n'est pas None
        elif "id" in selector and selector["id"] is not None:
            for id_name in selector["id"]:
                content = soup.find(tag, id=id_name)
                if content:
                    extracted_content[id_name] = content.get_text(strip=True)
    
    # Extrait spécifiquement les informations de prix si elles existent
    price_selectors = [
        {"tag": "span", "class": ["c-instrument--last", "price", "quote-price"]},
        {"tag": "div", "class": ["stock-price", "current-price"]}
    ]
    
    for selector in price_selectors:
        tag = selector.get("tag", "div")
        if "class" in selector and selector["class"] is not None:
            for class_name in selector["class"]:
                price_element = soup.find(tag, class_=class_name)
                if price_element:
                    extracted_content["price"] = price_element.get_text(strip=True)
                    break
    
    # Si rien n'a été trouvé, essayer d'extraire quelques éléments génériques
    if not extracted_content:
        # Chercher le titre principal
        title = soup.find("h1")
        if title:
            extracted_content["title"] = title.get_text(strip=True)
        
        # Chercher des éléments de classe contenant des mots-clés financiers
        financial_classes = ["price", "quote", "stock", "value", "cours"]
        for fc in financial_classes:
            elements = soup.find_all(class_=lambda c: c and fc in c.lower())
            for i, el in enumerate(elements):
                extracted_content[f"{fc}_{i}"] = el.get_text(strip=True)
    
    # Convertir le dictionnaire en texte formaté pour le retour
    formatted_text = ""
    for key, value in extracted_content.items():
        formatted_text += f"{key}: {value}\n\n"
    
    # Si toujours rien trouvé, retourner le texte brut
    return formatted_text if formatted_text else soup.get_text()

In [104]:
def load_any_website(url):
    """
    Charge les données de n'importe quel site web en adaptant l'extracteur
    en fonction du domaine.
    
    Args:
        url (str): L'URL de la page à scraper
        
    Returns:
        list: Liste des documents extraits
    """
    domain = urlparse(url).netloc
    
    # Choisir l'extracteur en fonction du domaine
    if "boursorama.com" in domain:
        # Utilisez l'extracteur spécifique à Boursorama
        extractor = boursorama_extractor
    elif any(x in domain for x in ["finance.yahoo.com", "investing.com"]):
        extractor = financial_site_extractor
    elif "github.io" in domain:
        # Extracteur pour les sites de documentation
        def doc_extractor(html):
            soup = BeautifulSoup(html, "lxml")
            content = soup.find("article", class_="md-content__inner")
            return content.get_text() if content else soup.get_text()
        extractor = doc_extractor
    else:
        # Extracteur générique par défaut
        extractor = lambda html: BeautifulSoup(html, "lxml").get_text()
    
    try:
        # Utiliser l'extracteur choisi
        loader = RecursiveUrlLoader(
            url,
            max_depth=1,
            extractor=extractor,
        )
        
        # Charger les documents
        docs = loader.load()
        return docs
    except Exception as e:
        print(f"Erreur lors du chargement du site {url}: {e}")
        # Méthode alternative en cas d'échec avec RecursiveUrlLoader
        try:
            response = requests.get(url)
            html_content = response.text
            extracted_text = extractor(html_content)
            
            # Créer un document similaire à celui que RecursiveUrlLoader aurait renvoyé
            from langchain.schema import Document
            doc = Document(page_content=extracted_text, metadata={"source": url})
            return [doc]
        except Exception as e2:
            print(f"Échec complet du scraping: {e2}")
            return []

# Exécution directe
url_to_scrape = "https://www.boursorama.com/cours/IQV/"
documents = load_any_website(url_to_scrape)

# MODIFICATION - Remplacez cette partie par le code ci-dessous
# if documents:
#     print("\nAperçu du contenu:")
#     print(documents[0].page_content[:500] + "...")
# else:
#     print("Aucun document extrait.")

# Afficher un aperçu du contenu extrait AVEC FORMATAGE
if documents:
    raw_content = documents[0].page_content
    
    # Afficher d'abord la version brute (optionnel)
    print("\nContenu brut extrait :")
    print(raw_content[:500] + "...")
    
    # Afficher la version formatée
    print("\nContenu formaté :")
    formatted_content = format_boursorama_output(raw_content)
    print(formatted_content)
else:
    print("Aucun document extrait.")


Contenu brut extrait :
stock_name:
CoursIQVIA HOLDINGS

main_content:
CoursActualitésAnalysesDurabilitéConsensusSociétéHistoriqueActionnairesCoursIQVIA HOLDINGS179,050USD-0,53%valeur indicative165,809
                                            EURUS46266C1053 IQVNYSE données temps différéPolitique d'exécutionCotation sur les autres placesAutres places de cotationFermerChargement...ouverture179,450clôture veille180,010+ haut180,705+ bas176,740volume252 566capital échangé0,14%valorisation31 569 MUSDcapi. boursièreCapit...

Contenu formaté :
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ IQVIA HOLDINGS - COTATION BOURSORAMA             ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

📈 COURS ACTUEL: 179,050 USD (-0,53%)
   Valeur indicative: 165,809 EUR

🏛️ RÉFÉRENCES:
   • Symbole: IQVIA
   • ISIN: US46266
   • Marché: NYSE (données temps différé)

📊 INDICATEURS DE MARCHÉ:
   • Ouverture: 179,450
   • Clôture veille: 180,010
   • Plus haut: 180,705
   • Plus bas: 176,740

📉 

In [105]:
# Exécution directe
url_to_scrape = "https://www.boursorama.com/cours/1rPCGM/"
documents = load_any_website(url_to_scrape)

# MODIFICATION - Remplacez cette partie par le code ci-dessous
# if documents:
#     print("\nAperçu du contenu:")
#     print(documents[0].page_content[:500] + "...")
# else:
#     print("Aucun document extrait.")

# Afficher un aperçu du contenu extrait AVEC FORMATAGE
if documents:
    raw_content = documents[0].page_content
    
    # Afficher d'abord la version brute (optionnel)
    print("\nContenu brut extrait :")
    print(raw_content[:500] + "...")
    
    # Afficher la version formatée
    print("\nContenu formaté :")
    formatted_content = format_boursorama_output(raw_content)
    print(formatted_content)
else:
    print("Aucun document extrait.")


Contenu brut extrait :
stock_name:
CoursCEGEDIM

main_content:
CoursActualitésAnalysesDurabilitéConsensusSociétéForumHistoriqueActionnairesCoursCEGEDIM13,3500EUR+0,75%indice de référenceCAC Mid & SmallFR0000053506 CGMEuronext Paris données temps réelPolitique d'exécutionCotation sur les autres placesAutres places de cotationFermerChargement...secteurServices informatiquesIndice de référenceCAC Mid & Smallouverture13,2500clôture veille13,2500+ haut13,3500+ bas13,2000volume2 597capital échangé0,02%valorisation188 MEURde...

Contenu formaté :
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ CEGEDIM HOLDINGS - COTATION BOURSORAMA             ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

📈 COURS ACTUEL:   (0,75%)
   Valeur indicative:  EUR

🏛️ RÉFÉRENCES:
   • Symbole: 
   • ISIN: US3
   • Marché:  (données temps différé)

📊 INDICATEURS DE MARCHÉ:
   • Ouverture: 13,2500
   • Clôture veille: 13,2500
   • Plus haut: 13,3500
   • Plus bas: 13,2000

📉 VOLUMES ET CAPITALISATION:
  